# Hyper Parameter Tuning
- hyper parameter: 모델 설정과 관련해 직접 지정할 수 있는 매개변수
- model parameter: 회귀계수(가중치), 절편 등 모델의 학습 대상이 되는 변수

- model parameter 에서는 매개변수라고 부를 수 있다.
- hyper parameter 는 우리가 값을 직접 지정해줄 수 있는 매개변수이면서도 모델의 설정에 직접적인 영향을 주는 매개변수이다.

**두 매개변수의 차이점은 모델의 학습 대상이 되는 변수인지 아닌지 차이이다.**

### 하이퍼 파라미터 vs 모델 파라미터: 핵심 요약


| 비교 항목 | 하이퍼 파라미터(Hyper Parameter) | 모델 파라미터(Model Parameter) |
|-------|-------|-------|
| 결정 주체 | **사용자(개발자)**가 직접 설정. | 모델이 데이터를 통해 스스로 학습. |
| 결정 시점	| 모델 학습 전에 설정. |	모델 학습 과정에서 자동으로 결정. |
| 역할 | 학습 과정의 전반적인 동작 방식과 구조를 제어. | 입력 데이터로부터 예측을 수행하는 모델의 내부 변수. |
| 예시 | K (KNN의 이웃 수), learning rate, n_estimators 등. | 회귀 계수(가중치), 절편(편향), 신경망의 가중치 등. |

| 비교 항목 |	cross_validate |	GridSearchCV |
|---|---|---|
|목적|	단일 모델의 일반화 성능을 평가합니다. 데이터를 여러 겹(fold)으로 나누어 교차 검증을 수행하고, 각 폴드에서 모델을 훈련 및 평가한 결과를 반환합니다.|	최적의 하이퍼파라미터를 탐색합니다. 여러 하이퍼파라미터 조합을 시도하고, 각 조합에 대해 교차 검증을 수행하여 가장 좋은 성능을 내는 조합을 찾습니다.|
|모델 학습|	교차 검증 과정에서 모델을 여러 번 학습합니다. 예를 들어, 5-폴드 교차 검증을 사용하면, 모델은 5번 훈련됩니다.|	탐색 과정에서 모델을 여러 번 학습합니다. 예를 들어, 하이퍼파라미터 조합이 10개이고 5-폴드 교차 검증을 사용하면, 총 50번 모델이 훈련됩니다.|
|반환값|	각 폴드에서의 훈련 및 평가 점수, 학습 시간, 예측 시간 등 성능 지표를 담은 딕셔너리를 반환합니다.|	최적의 하이퍼파라미터 조합(best_params_)과 그 조합으로 얻은 최고 성능 점수(best_score_)를 포함하는 객체를 반환합니다.|
|모델 변화|	결과적으로 하나의 모델이 만들어지거나 튜닝되지는 않습니다. 단순히 특정 하이퍼파라미터로 설정된 모델의 성능을 평가하는 용도입니다.|	최적의 하이퍼파라미터로 재훈련된 모델(best_estimator_)을 얻을 수 있습니다. (refit=True일 때) 이 모델은 전체 훈련 데이터를 사용하여 학습된 최종 모델입니다.|

### GridSearchCV

In [1]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# 최적의 k값 하이퍼 파라미터를 찾기 위해서 GridSearchCV 사용

# 데이터 로드
iris_input, iris_target = load_iris(return_X_y=True)

knn = KNeighborsClassifier()

# 하이퍼 파라미터 탐색 범위 설정, 모든 조합에 대해서 교차 검증 (CV)를 수행해주는 함수

# 첫번째 인자: 최적화할 모델
# 두번째 인자: 테스트할 파라미터 (딕셔너리),, 탐색할 하이퍼 파라미터 범위
# 예) param_grid = {'n_neighbors': [1, 3, 5, 7, 9]}

# scoring: 평가 지표 (accuracy, precision, recall, f1) → 분류 모델에서 사용이기 때문에 설저안 평가지표는 분류 모델에서만 사용 가능
# 세번째 인자: cv → 교차 검증 횟수

# 테스트할 파라미터 값
params = {
    'n_neighbors': range(1, 13, 2) # → 키값 : 모델의 하이퍼파라미터 이름, 값 : 모델의 하이퍼파라미터 값
    # 1부터 13까지 2씩 증가하는 범위

}

grid = GridSearchCV(knn, params, scoring='accuracy', cv=5) # GridSearchCV 는 교차 검증을 수행하는 모델이지만, 최적의 하이퍼파라미터를 찾는 모델이다.
                                                            # 그래서 X, y 데이터를 넣어줄 필요가 없다.
grid.fit(iris_input, iris_target) # fit 메서드는 교차 검증을 수행하는 메서드이다. 여기서는 학습의 과정이 아니라 최적의 하이퍼파라미터를 찾는 과정이다.

# 최적의 파라미터와 최고의 정확도를 출력
print("최적의 파라미터             :", grid.best_params_)
print("최적의 파라미터로 학습된 모델:", grid.best_estimator_)
print("최적화된 정확도 점수        :", grid.best_score_) #

최적의 파라미터             : {'n_neighbors': 7}
최적의 파라미터로 학습된 모델: KNeighborsClassifier(n_neighbors=7)
최적화된 정확도 점수        : 0.9800000000000001


In [2]:
# 최적의 파라미터로 학습된 모델 저장
# joblib.dump(best_knn, 'best_knn_model.pkl')

# 저장된 모델 불러오기

best_knn = grid.best_estimator_
best_knn.fit(iris_input, iris_target)
best_knn.score(iris_input, iris_target)
# 왜 gird.fit() 에서 출력되는 최적화된 정확도 점수, grid.best_score_ 에서 출력되는 최적화된 정확도(best_knn.score()) 점수가 다른지 알아보자.
# 최적화된 정확도 점수는 교차 검증을 수행한 후 최적의 파라미터로 학습된 모델의 평균 정확도 점수이다.


best_knn.predict(iris_input)

# → .predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [3]:
best_knn = grid.best_estimator_
best_knn.fit(iris_input, iris_target)
best_knn.score(iris_input, iris_target)

print("best_knn 모델 객체   : ", best_knn)
print("best_knn 모델 정확도 : ", best_knn.score(iris_input, iris_target))


# 최적의 파라미터로 학습된 모델 저장
# joblib.dump(best_knn, 'best_knn_model.pkl')

# 저장된 모델 불러오기


best_knn 모델 객체   :  KNeighborsClassifier(n_neighbors=7)
best_knn 모델 정확도 :  0.9733333333333334


### RandomSearchCV
- 하이퍼 파라미터의 값 목록이나 값의 범위를 제공하는데, 이 범위 중에 랜덤하게 값을 뽑아내 최적의 하이퍼 파라미터 조합을 찾는다.
    - 탐색범위가 넓을 때 짧은 시간 내에 좋은 결과를 얻을 수 있다.
    - 랜덤하게 값을 추출해 계산하므로, 전역 최적값을 놓칠 수 있다.

In [4]:
from sklearn.model_selection import RandomizedSearchCV

# 모델 생성
knn = KNeighborsClassifier()

# 테스트할 파라미터 범위 설정
params = {
    'n_neighbors': range(1, 1000, 2)
}

# n_iter: 탐색할 최적의 하이퍼 파라미터 조합의 개수 (기본값: 10)
    # 너무 작게 설정하면 전역 최적값을 놓칠 수 있다. = 랜덤하게 값을 뽑아내기 때문에 전역 최적값을 놓칠 수 있다.
    # = 좋은 조합을 찾을 가능성 저하

    # 너무 크게 설정하면 시간이 오래 걸린다.

rd_search = RandomizedSearchCV(knn, params, scoring='accuracy', cv=5, n_iter=10, random_state=0) # , random_state=0
rd_search.fit(iris_input, iris_target)

print('rd_search 최고의 파라미터로 학습된 모델       : ', rd_search.best_estimator_)
print('rd_search 최고의 파라미터                    : ', rd_search.best_params_)
print('rd_search 최고의 파라미터로 학습된 모델 정확도 : ', rd_search.best_score_)
# rd_search.predict(iris_input)

# rd_search.cv_results_

# {'mean_fit_time': array([0.        , 0.00080833, 0.00035872, 0.        , 0.        ,
#         0.        , 0.00289865, 0.        , 0.        , 0.00039992]),
#  'std_fit_time': array([0.        , 0.00099001, 0.00050749, 0.        , 0.        ,
#         0.        , 0.00579729, 0.        , 0.        , 0.00048991]),
#  'mean_score_time': array([0.00151343, 0.00036316, 0.00047355, 0.        , 0.00276527,
#         0.        , 0.        , 0.00100398, 0.00099759, 0.00020022]),
#  'std_score_time': array([2.16472652e-03, 7.26318359e-04, 5.92680092e-04, 0.00000000e+00,
#         4.74616461e-03, 0.00000000e+00, 0.00000000e+00, 1.09059236e-05,
#         3.60445999e-06, 4.00447845e-04]),
#  'param_n_neighbors': masked_array(data=[181, 509, 567, 891, 923, 31, 633, 979, 319, 307],
#               mask=[False, False, False, False, False, False, False, False,
#                     False, False],
#         fill_value=999999),

# 탐색한 파라미터 값, 총 n_iter 개수만큼 탐색 : 10개

#  'params': [{'n_neighbors': 181},
#   {'n_neighbors': 509},
#   {'n_neighbors': 567},
#   {'n_neighbors': 891},
#   {'n_neighbors': 923},
#   {'n_neighbors': 31},
#   {'n_neighbors': 633},
#   {'n_neighbors': 979},
#   {'n_neighbors': 319},
#   {'n_neighbors': 307}],
#  'split0_test_score': array([nan, nan, nan, nan, nan, 0.9, nan, nan, nan, nan]),
#  'split1_test_score': array([       nan,        nan,        nan,        nan,        nan,
#         0.96666667,        nan,        nan,        nan,        nan]),
#  'split2_test_score': array([nan, nan, nan, nan, nan, 0.9, nan, nan, nan, nan]),
#  'split3_test_score': array([nan, nan, nan, nan, nan, 0.9, nan, nan, nan, nan]),
#  'split4_test_score': array([nan, nan, nan, nan, nan,  1., nan, nan, nan, nan]),
#  'mean_test_score': array([       nan,        nan,        nan,        nan,        nan,
#         0.93333333,        nan,        nan,        nan,        nan]),
#  'std_test_score': array([      nan,       nan,       nan,       nan,       nan, 0.0421637,
#               nan,       nan,       nan,       nan]),
#  'rank_test_score': array([2, 2, 2, 2, 2, 1, 2, 2, 2, 2], dtype=int32)}

rd_search 최고의 파라미터로 학습된 모델       :  KNeighborsClassifier(n_neighbors=31)
rd_search 최고의 파라미터                    :  {'n_neighbors': 31}
rd_search 최고의 파라미터로 학습된 모델 정확도 :  0.9333333333333332


c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\model_selection\_validation.py:953: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\model_selection\_validation.py", line 942, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\metrics\_scorer.py", line 308, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\metrics\_scorer.py", line 400, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "c:\Users\Playdata\anacond

---

### HyperOpt

**hyper.hp클래스**
<table border="1">
  <thead>
    <tr>
      <th>함수명</th>
      <th>설명</th>
      <th>사용 방법</th>
      <th>예시 코드</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>hp.uniform</td>
      <td>연속적인 실수 값 샘플링</td>
      <td>hp.uniform(label, low, high)</td>
      <td><code>hp.uniform('learning_rate', 0.01, 0.1)</code></td>
    </tr>
    <tr>
      <td>hp.quniform</td>
      <td>연속적이지만 일정 간격(q)을 갖는 값 샘플링</td>
      <td>hp.quniform(label, low, high, q)</td>
      <td><code>hp.quniform('num_layers', 1, 5, 1)</code></td>
    </tr>
    <tr>
      <td>hp.loguniform</td>
      <td>로그 스케일로 분포된 실수 값 샘플링</td>
      <td>hp.loguniform(label, low, high)</td>
      <td><code>hp.loguniform('reg_param', -3, 0)</code></td>
    </tr>
    <tr>
      <td>hp.randint</td>
      <td>정수 값 샘플링</td>
      <td>hp.randint(label, upper)</td>
      <td><code>hp.randint('num_trees', 1, 100)</code></td>
    </tr>
    <tr>
      <td>hp.choice</td>
      <td>주어진 리스트 중 임의의 값 샘플링</td>
      <td>hp.choice(label, options)</td>
      <td><code>hp.choice('optimizer', ['adam', 'sgd', 'rmsprop'])</code></td>
    </tr>
    <tr>
      <td>hp.normal</td>
      <td>정규분포에서 값 샘플링</td>
      <td>hp.normal(label, mean, std)</td>
      <td><code>hp.normal('dropout_rate', 0.3, 0.05)</code></td>
    </tr>
    <tr>
      <td>hp.lognormal</td>
      <td>로그 정규분포에서 값 샘플링</td>
      <td>hp.lognormal(label, mean, std)</td>
      <td><code>hp.lognormal('scale', 0, 1)</code></td>
    </tr>
  </tbody>
</table>

In [5]:
!pip install hyperopt

In [6]:
from hyperopt import hp

# 여러개의 하이퍼 파라미터의 탐색을 하기 위해서는 딕셔너리 형태로 탐색 공간을 정의해야 한다.
# 'X' 와 'y'는 하이퍼 파라미터 명
# hp.quniform('X', -10, 10, 1) 은 'X' 하이퍼 파라미터의 탐색 공간을 -10 부터 10 까지 1 의 간격으로 설정한다.
# hp.quniform('y', -15, 15, 2) 은 'y' 하이퍼 파라미터의 탐색 공간을 -15 부터 15 까지 2 의 간격으로 설정한다.

# 검색 공간          

# '파라미터 명': hp.quniform('파라미터 명', 탐색 공간 하한, 탐색 공간 상한, 탐색 공간 간격)
search_space = {
    'X': hp.quniform('X', -10, 10, 1), # 연속적인 실수 값 샘플링
    'y': hp.quniform('y', -15, 15, 2)  # 연속적이지만 일정 간격(q)을 갖는 값 샘플링
}

In [7]:
import hyperopt

# 목적 함수 (objective function)
# search_space 는 탐색 공간을 정의한 딕셔너리를 매개변수로 넣어줌.
def objective(search_space):
    X = search_space['X']
    y = search_space['y']

    return {
        'loss': X**2 + 20 * y,
        'status': hyperopt.STATUS_OK
    }

In [8]:
from hyperopt import fmin, tpe, Trials

# 하이퍼 파라미터를 탐색하는 과정을 저장하는 객체
trials = Trials()

# fmin() 함수는 최소화 문제를 해결하는 함수이다. → 목적 함수의 최소값을 찾는 함수수
# fn: 최적화 파라미터를 찾기 위한 목적 함수
# space: 탐색 공간
# algo: 탐색 알고리즘
# max_evals: 탐색 횟수
# trials: 탐색 과정을 저장하는 객체
best_val = fmin(
    fn=objective,
    space=search_space, # 검색 공간
    algo=tpe.suggest,   # 베이지안 최적화 알고리즘 적용
    max_evals=500,      # 탐색 횟수
    trials=trials       # 탐색 과정을 저장하는 객체
)

best_val

100%|██████████| 500/500 [00:06<00:00, 80.07trial/s, best loss: -280.0] 


{'X': np.float64(-0.0), 'y': np.float64(-14.0)}

In [9]:
trials.results # 탐색 과정을 저장한 객체, max_evals 만큼 탐색 과정을 저장한다. 500번 탐색 과정을 저장한다.

# 탐색과정 -> 목적함수 반환값 (loss와 실행 상태 저장)


[{'loss': -216.0, 'status': 'ok'},
 {'loss': 44.0, 'status': 'ok'},
 {'loss': -151.0, 'status': 'ok'},
 {'loss': 176.0, 'status': 'ok'},
 {'loss': 129.0, 'status': 'ok'},
 {'loss': 201.0, 'status': 'ok'},
 {'loss': 144.0, 'status': 'ok'},
 {'loss': -56.0, 'status': 'ok'},
 {'loss': -159.0, 'status': 'ok'},
 {'loss': -96.0, 'status': 'ok'},
 {'loss': -76.0, 'status': 'ok'},
 {'loss': -224.0, 'status': 'ok'},
 {'loss': 96.0, 'status': 'ok'},
 {'loss': 36.0, 'status': 'ok'},
 {'loss': -36.0, 'status': 'ok'},
 {'loss': -96.0, 'status': 'ok'},
 {'loss': 316.0, 'status': 'ok'},
 {'loss': 24.0, 'status': 'ok'},
 {'loss': -239.0, 'status': 'ok'},
 {'loss': -76.0, 'status': 'ok'},
 {'loss': -180.0, 'status': 'ok'},
 {'loss': -236.0, 'status': 'ok'},
 {'loss': -236.0, 'status': 'ok'},
 {'loss': -276.0, 'status': 'ok'},
 {'loss': -255.0, 'status': 'ok'},
 {'loss': -175.0, 'status': 'ok'},
 {'loss': -180.0, 'status': 'ok'},
 {'loss': -104.0, 'status': 'ok'},
 {'loss': -180.0, 'status': 'ok'},
 {'l

In [10]:
# 탐색 과정을 저장한 객체의 결과를 출력
for trial in trials.trials:
    print(trial)


{'state': 2, 'tid': 0, 'spec': None, 'result': {'loss': -216.0, 'status': 'ok'}, 'misc': {'tid': 0, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'X': [np.int64(0)], 'y': [np.int64(0)]}, 'vals': {'X': [np.float64(8.0)], 'y': [np.float64(-14.0)]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2025, 9, 25, 7, 55, 8, 399000), 'refresh_time': datetime.datetime(2025, 9, 25, 7, 55, 8, 399000)}
{'state': 2, 'tid': 1, 'spec': None, 'result': {'loss': 44.0, 'status': 'ok'}, 'misc': {'tid': 1, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'X': [np.int64(1)], 'y': [np.int64(1)]}, 'vals': {'X': [np.float64(2.0)], 'y': [np.float64(2.0)]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2025, 9, 25, 7, 55, 8, 400000), 'refresh_time': datetime.datetime(2025, 9, 25, 7, 55, 8, 400000)}
{'state': 2, 'tid': 2, 'spec': None, 'result': {'loss': -151.0, 'status': 'ok'}, 'misc': {'tid': 2, 

In [11]:
# 탐색과정 -> 하이퍼 파라미터값을 딕셔너리(리스트) 형태로 저장
trials.vals # 탐색 과정동안 탐색한 파라미터 X, y 의 값을 저장한 딕셔너리

{'X': [np.float64(8.0),
  np.float64(2.0),
  np.float64(-7.0),
  np.float64(4.0),
  np.float64(7.0),
  np.float64(-1.0),
  np.float64(8.0),
  np.float64(-8.0),
  np.float64(-1.0),
  np.float64(8.0),
  np.float64(-2.0),
  np.float64(-4.0),
  np.float64(-4.0),
  np.float64(6.0),
  np.float64(-2.0),
  np.float64(8.0),
  np.float64(-6.0),
  np.float64(-8.0),
  np.float64(-1.0),
  np.float64(-2.0),
  np.float64(-10.0),
  np.float64(2.0),
  np.float64(2.0),
  np.float64(2.0),
  np.float64(5.0),
  np.float64(5.0),
  np.float64(10.0),
  np.float64(4.0),
  np.float64(10.0),
  np.float64(1.0),
  np.float64(3.0),
  np.float64(6.0),
  np.float64(1.0),
  np.float64(3.0),
  np.float64(5.0),
  np.float64(9.0),
  np.float64(0.0),
  np.float64(4.0),
  np.float64(7.0),
  np.float64(1.0),
  np.float64(-5.0),
  np.float64(6.0),
  np.float64(3.0),
  np.float64(-3.0),
  np.float64(9.0),
  np.float64(0.0),
  np.float64(7.0),
  np.float64(5.0),
  np.float64(7.0),
  np.float64(-3.0),
  np.float64(4.0),
  np.fl

- hyperopt를 활용한 XGBoost 하이퍼 파라미터 튜닝

In [12]:
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score

# 데이터 로드
data = load_breast_cancer()

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    data.data,
    data.target,
    random_state=42
)

xgb_clf = XGBClassifier()

# 1. 검색 공간 -> 최적의 하이파라미터를 찾아야하므로 key 값으로 하이퍼 파라미터 명을 넣어준다.

# n_estimators: 트리 개수
# max_depth: 트리 깊이
# learning_rate: 학습률
# colsample_bytree: Tree를 생성할 때 샘플링 비율을 조절하는 하이퍼 파라미터 -> 라벨이 아니고 특성 샘플링 비율 -> 실수값이기 떄문에 hp.uniform 사용
# subsample: 데이터 샘플링 비율
# gamma: 최소 분할 노드 수

search_space = {
    'n_estimators': hp.quniform('n_estimators', 100, 500, 100),
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2), # 실수값 샘플링, 0.01 ~ 0.2 사이의 값 추출, hp.uniform(label, low, high)
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1), # 여러개의 특성중에 랜덤하게 샘플링 비율을 조절하는 하이퍼 파라미터, 
                                                                # 50% ~ 100% 사이의 값 추출
    #'subsample': hp.uniform('subsample', 0.5, 1),
    #'gamma': hp.uniform('gamma', 0, 1),
}

# 2. 목적 함수
def xgb_objective(search_space):
    xgb_clf = XGBClassifier(
        n_estimators=int(search_space['n_estimators']),
        max_depth=int(search_space['max_depth']),
        learning_rate=search_space['learning_rate'],
        colsample_bytree=search_space['colsample_bytree'],
        #subsample=search_space['subsample'],
        #gamma=search_space['gamma'],
    )

    mean_acc = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3).mean()

    return {
        'loss': -1 * mean_acc,
        # 1 - mean_acc 은 정확도를 나타내는 값이다. 
        # 정확도가 높을수록 좋은 모델이므로 최소화 문제를 해결하기 위해 -1 * mean_acc 를 사용한다.
        # 최소화 문제를 해결하기 위해 -1 * mean_acc 를 사용한다. why? fmin() 함수는 최소화 문제를 해결하는 함수이기 때문이다.
        'status': hyperopt.STATUS_OK
    }

# 3. Trials() + fmin()
trials = Trials()

best = fmin(
    fn=xgb_objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

# best loss: -0.971830985915493 -> 정확도가 97% 이상이다. (-1*mean_acc)


100%|██████████| 50/50 [00:17<00:00,  2.86trial/s, best loss: -0.971830985915493] 


In [13]:
# 탐색 과정을 저장한 객체의 결과를 출력
for trial in trials.trials:
    print(trial)


{'state': 2, 'tid': 0, 'spec': None, 'result': {'loss': -0.9624413145539905, 'status': 'ok'}, 'misc': {'tid': 0, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'colsample_bytree': [np.int64(0)], 'learning_rate': [np.int64(0)], 'max_depth': [np.int64(0)], 'n_estimators': [np.int64(0)]}, 'vals': {'colsample_bytree': [np.float64(0.9277931089889639)], 'learning_rate': [np.float64(0.17263790146238941)], 'max_depth': [np.float64(10.0)], 'n_estimators': [np.float64(300.0)]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2025, 9, 25, 7, 55, 14, 774000), 'refresh_time': datetime.datetime(2025, 9, 25, 7, 55, 15, 74000)}
{'state': 2, 'tid': 1, 'spec': None, 'result': {'loss': -0.9624413145539906, 'status': 'ok'}, 'misc': {'tid': 1, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'colsample_bytree': [np.int64(1)], 'learning_rate': [np.int64(1)], 'max_depth': [np.int64(1)], 'n_estimators': [np.int64(1)]}, 'vals':

In [14]:
trials.best_trial

{'state': 2,
 'tid': 36,
 'spec': None,
 'result': {'loss': -0.971830985915493, 'status': 'ok'},
 'misc': {'tid': 36,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'colsample_bytree': [36],
   'learning_rate': [36],
   'max_depth': [36],
   'n_estimators': [36]},
  'vals': {'colsample_bytree': [np.float64(0.5311768087781208)],
   'learning_rate': [np.float64(0.16582728359839924)],
   'max_depth': [np.float64(10.0)],
   'n_estimators': [np.float64(200.0)]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2025, 9, 25, 7, 55, 28, 478000),
 'refresh_time': datetime.datetime(2025, 9, 25, 7, 55, 28, 716000)}

---

### Optuna

<table border="1">
    <thead>
        <tr>
            <th>함수명</th>
            <th>설명</th>
            <th>사용 방법</th>
            <th>예시 코드</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>suggest_uniform</td>
            <td>연속적인 실수 값 샘플링</td>
            <td>trial.suggest_uniform(name, low, high)</td>
            <td><code>trial.suggest_uniform('learning_rate', 0.01, 0.1)</code></td>
        </tr>
        <tr>
            <td>suggest_discrete_uniform</td>
            <td>연속적이지만 일정 간격(step)을 갖는 값 샘플링</td>
            <td>trial.suggest_discrete_uniform(name, low, high, step)</td>
            <td><code>trial.suggest_discrete_uniform('num_layers', 1, 5, 1)</code></td>
        </tr>
        <tr>
            <td>suggest_loguniform</td>
            <td>로그 스케일로 분포된 실수 값 샘플링</td>
            <td>trial.suggest_loguniform(name, low, high)</td>
            <td><code>trial.suggest_loguniform('reg_param', 1e-3, 1)</code></td>
        </tr>
        <tr>
            <td>suggest_int</td>
            <td>정수 값 샘플링</td>
            <td>trial.suggest_int(name, low, high, step)</td>
            <td><code>trial.suggest_int('num_trees', 1, 100)</code></td>
        </tr>
        <tr>
            <td>suggest_categorical</td>
            <td>주어진 리스트 중 임의의 값 샘플링</td>
            <td>trial.suggest_categorical(name, choices)</td>
            <td><code>trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop'])</code></td>
        </tr>
        <tr>
            <td>suggest_float</td>
            <td>연속적인 실수 값 샘플링 (<code>step</code> 사용 가능)</td>
            <td>trial.suggest_float(name, low, high, step=None, log=False)</td>
            <td><code>trial.suggest_float('alpha', 0.1, 1.0, step=0.1)</code></td>
        </tr>
    </tbody>
</table>

In [15]:
!pip install optuna

##### Optuna는 trial 객체를 만들어서 매서드를 통해서 하이퍼 파라미터를 탐색
##### 개별적인 시도를 하기 위해서는 trial 객체를 만들어야 한다.

In [16]:
import optuna

# 목적 함수
def objective(trial):
    X = trial.suggest_uniform('X', -10, 10)
    y = trial.suggest_uniform('y', -15, 15)

    return (X - 3) ** 2 + (y + 5) ** 2

# 스터디 생성
# study = optuna에서 최적화 결과를 저장하는 객체
# direction='minimize' 은 최소화 문제를 해결하는 것을 의미 -> return 값이 작을수록 좋은 결과 -> 최소화 방향으로 학습이 진행될 예정
study = optuna.create_study(direction='minimize')

# 최적화 실행
study.optimize(objective, n_trials=500)

print(study)
# 최적화 결과 출력
# print(study.best_trial)
# 최적화 결과 확인
print('==='*60)
print('최적화 결과 확인', study.best_trial.params)
print('==='*60)
print('최적화 결과 확인', study.best_trial.value)
print('==='*60)



[I 2025-09-25 16:55:35,589] A new study created in memory with name: no-name-63cf3075-a7c5-4292-9acc-26c90562ee78
C:\Users\Playdata\AppData\Local\Temp\ipykernel_34480\3402769400.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  X = trial.suggest_uniform('X', -10, 10)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_34480\3402769400.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  y = trial.suggest_uniform('y', -15, 15)
[I 2025-09-25 16:55:35,593] Trial 0 finished with value: 118.95118756643888 and parameters: {'X': -5.904765852468712, 'y': -11.297327423530243}. Best is trial 0 with value: 118.95118756643888.
[I 2025-09-25 16:55:35,595] Trial 1 finished with value: 447.3248276453587 and parameters: {'X'

최적화 결과 확인 {'X': 2.998623511181959, 'y': -5.00162628227869}
최적화 결과 확인 4.53951551617394e-06


In [17]:
print(study.best_trial)

FrozenTrial(number=251, state=1, values=[4.53951551617394e-06], datetime_start=datetime.datetime(2025, 9, 25, 16, 55, 37, 103971), datetime_complete=datetime.datetime(2025, 9, 25, 16, 55, 37, 112530), params={'X': 2.998623511181959, 'y': -5.00162628227869}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'X': FloatDistribution(high=10.0, log=False, low=-10.0, step=None), 'y': FloatDistribution(high=15.0, log=False, low=-15.0, step=None)}, trial_id=251, value=None)


In [18]:
# 최적화 결과 확인
print('최적화 결과 확인', study.best_trial.params)
print('최적화 결과 확인', study.best_trial.value)



최적화 결과 확인 {'X': 2.998623511181959, 'y': -5.00162628227869}
최적화 결과 확인 4.53951551617394e-06


In [19]:
!pip install plotly

In [20]:
# 바로 위 셀에서 !pip install plotly로 plotly를 설치했지만,
# 주피터 노트북에서는 셀마다 커널이 분리되어 있거나, 설치 직후 import 시 커널 재시작이 필요할 수 있습니다.
# plotly 설치 후, 커널을 재시작하거나, 설치 셀을 실행한 뒤 다시 import해야 정상적으로 동작합니다.

import optuna.visualization as vis
import plotly

# plotly 라이브러리 : 동적 시각화 라이브러리

# 파라미터 중요도 시각화
vis.plot_param_importances(study).show()
# 최적화 과정 시각화
vis.plot_optimization_history(study).show()
# 병렬 좌표 시각화
vis.plot_parallel_coordinate(study).show()

- optuna를 활용한 XGBoost 하이퍼 파라미터 튜닝

In [ ]:
# optuna를 활용한 XGBoost 하이퍼 파라미터 튜닝
import optuna
from xgboost import XGBClassifier

# 1. 목적 함수 → trial 객체를 매개변수로 받아서 하이퍼 파라미터를 탐색, hyperopt는 hp 클래스를 통해서 하이퍼 파라미터를 탐색

# 왜 오류가 나는지 살펴보자.
# 아래 코드에서 목적 함수 이름이 xgb_optuna_objective인데,
# study.optimize(xgb_objective, n_trials=50)로 잘못 호출하고 있다.
# 함수 이름이 일치하지 않아서 NameError가 발생한다.

# TypeError: 'Trial' object is not subscriptable 오류는 trial['param']처럼 trial 객체를 딕셔너리처럼 인덱싱해서 발생합니다.
# optuna의 trial 객체는 딕셔너리처럼 사용하지 않고, trial.suggest_xxx() 메서드로 파라미터를 제안받아야 합니다.
# 아래는 올바른 예시입니다.

def xgb_optuna_objective(trial):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }

    xgb_clf = XGBClassifier(**params) # 키, 벨류 형태로 넣어준다. → 언팩킹

    return cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3).mean()
    # 교차 검증을 수행한 후 최적의 파라미터로 학습된 모델의 평균 정확도 점수를 반환 → 최대화 문제를 해결하는 것을 의미

# 2. study 객체 -> 최적화
study = optuna.create_study(direction='maximize') # → 최대화 문제를 해결하는 것을 의미, 목적함수가 최대화 문제를 해결하는 것을 의미

# 목적 함수 이름을 올바르게 넣어야 한다!

study.optimize(xgb_optuna_objective, n_trials=50)  # (O) 올바른 함수명

# 4. 결과 출력
print(study)
print('==='*60)
print('최적화 결과 확인', study.best_trial.params)
print('==='*60)
print('최적화 결과 확인', study.best_trial.value)
print('==='*60)


[I 2025-09-25 17:05:57,080] A new study created in memory with name: no-name-19381dc1-63fd-49ad-8ad7-f812c5db02ae
C:\Users\Playdata\AppData\Local\Temp\ipykernel_34480\3471382753.py:18: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v3.5.0 for details.

[I 2025-09-25 17:05:57,485] Trial 0 finished with value: 0.960093896713615 and parameters: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.047454778362726, 'colsample_bytree': 0.9114885549350937}. Best is trial 0 with value: 0.960093896713615.
C:\Users\Playdata\AppData\Local\Temp\ipykernel_34480\3471382753.py:18: FutureWarning:

suggest_int() got {'step'} as positio

최적화 결과 확인 {'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.19905346089916254, 'colsample_bytree': 0.5065038951733922}
최적화 결과 확인 0.971830985915493


#### HyperOpt vs Optuna

In [25]:
from sklearn.metrics import accuracy_score

xgb_hpopt = XGBClassifier(
    n_estimators=36,
    max_depth=10,
    learning_rate=0.16582728359839924,
    colsample_bytree=0.5311768087781208
)

xgb_optuna = XGBClassifier(
    n_estimator=400,
    max_depth=5,
    learning_rate=0.19905346089916254,
    colsample_bytree=0.5065038951733922
)

xgb_hpopt.fit(X_train, y_train)
xgb_optuna.fit(X_train, y_train)

hpopt_pred = xgb_hpopt.predict(X_test)
optuna_pred = xgb_optuna.predict(X_test)

print(f"HyperOpt 최적 파라미터 적용 : {accuracy_score(y_test, hpopt_pred)}")
print(f"Optuna   최적 파라미터 적용 : {accuracy_score(y_test, optuna_pred)}")

HyperOpt 최적 파라미터 적용 : 0.958041958041958
Optuna   최적 파라미터 적용 : 0.958041958041958


c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\xgboost\training.py:183: UserWarning:

[17:10:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_estimator" } are not used.




In [ ]:
# Optuna 로 나온 최적의 결과 파라미터 값

최적화 결과 확인 {'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.19905346089916254, 'colsample_bytree': 0.5065038951733922}
====================================================================================================================================================================================
최적화 결과 확인 0.971830985915493

# HyperOpt 로 나온 최적의 결과 파라미터 값

{'state': 2,
 'tid': 36,
 'spec': None,
 'result': {'loss': -0.971830985915493, 'status': 'ok'},
 'misc': {'tid': 36,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'colsample_bytree': [36],
   'learning_rate': [36],
   'max_depth': [36],
   'n_estimators': [36]},
  'vals': {'colsample_bytree': [np.float64(0.5311768087781208)],
   'learning_rate': [np.float64(0.16582728359839924)],
   'max_depth': [np.float64(10.0)],
   'n_estimators': [np.float64(200.0)]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2025, 9, 25, 7, 55, 28, 478000),
 'refresh_time': datetime.datetime(2025, 9, 25, 7, 55, 28, 716000)}
 

In [23]:
import optuna
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split

# 데이터
X, y = load_breast_cancer(return_X_y=True)

# 목적 함수 (trial 사용, suggest_* 사용)
def xgb_optuna_objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500, step=100),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        # 안정적으로 돌기 위한 옵션들(환경에 맞게 조절 가능)
        "n_jobs": -1,
        "tree_method": "hist",   # GPU 없을 때도 빠르게
        "random_state": 42,
        "eval_metric": "logloss" # 최신 xgboost에서 경고 방지
    }

    model = XGBClassifier(**params)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(model, X, y, scoring="accuracy", cv=cv).mean()
    return score  # maximize

# 스터디 생성 & 최적화
study = optuna.create_study(direction="maximize")
study.optimize(xgb_optuna_objective, n_trials=50)

print("Best params:", study.best_trial.params)
print("Best value:", study.best_trial.value)


[I 2025-09-25 17:04:14,477] A new study created in memory with name: no-name-612dffe1-8c57-4c0b-b4cc-0230bbcaf0bb
[I 2025-09-25 17:04:14,712] Trial 0 finished with value: 0.9596027104799033 and parameters: {'n_estimators': 200, 'max_depth': 8, 'learning_rate': 0.14491289509986163, 'colsample_bytree': 0.502414741275466}. Best is trial 0 with value: 0.9596027104799033.
[I 2025-09-25 17:04:14,900] Trial 1 finished with value: 0.9560753736192332 and parameters: {'n_estimators': 100, 'max_depth': 8, 'learning_rate': 0.08708686854252205, 'colsample_bytree': 0.9298022883367485}. Best is trial 0 with value: 0.9596027104799033.
[I 2025-09-25 17:04:15,148] Trial 2 finished with value: 0.9631021999443052 and parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.18106494981613017, 'colsample_bytree': 0.5543614333285689}. Best is trial 2 with value: 0.9631021999443052.
[I 2025-09-25 17:04:15,284] Trial 3 finished with value: 0.9543117051888982 and parameters: {'n_estimators': 100, '

Best params: {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.19883797062281633, 'colsample_bytree': 0.6599629908689308}
Best value: 0.9666295368049754
